In [1]:
import os

os.environ['KAGGLE_USERNAME'] = 'yannspf'
os.environ['KAGGLE_KEY'] = '47e6ebd956faccc933d2bba9d554bbbe'
try:
    import kaggle
except ImportError:
    os.system('pip install kaggle')

In [2]:
import kagglehub

# Billboard Hot 100 dataset
path1 = kagglehub.dataset_download("thedevastator/billboard-hot-100-audio-features")
print("Path to Billboard dataset:", path1)

# Spotify Top Songs dataset
path2 = kagglehub.dataset_download("asaniczka/top-spotify-songs-in-73-countries-daily-updated")
print("Path to Spotify dataset:", path2)

Path to Billboard dataset: /Users/yann/.cache/kagglehub/datasets/thedevastator/billboard-hot-100-audio-features/versions/1


100%|██████████| 154M/154M [00:08<00:00, 20.1MB/s] 

Extracting files...


Path to Spotify dataset: /Users/yann/.cache/kagglehub/datasets/asaniczka/top-spotify-songs-in-73-countries-daily-updated/versions/580


In [3]:
import pandas as pd

# Load and preview Billboard datasets
audio_features_df = pd.read_csv(os.path.join(path1, "Hot 100 Audio Features.csv"))
hot_stuff_df = pd.read_csv(os.path.join(path1, "Hot Stuff.csv"))

# Load and preview Spotify dataset
spotify_df = pd.read_csv(os.path.join(path2, "universal_top_spotify_songs.csv"))

# Print first 5 rows of each dataset
print(" First 5 rows of 'Hot 100 Audio Features.csv':")
print(audio_features_df['spotify_genre'].head())

 First 5 rows of 'Hot 100 Audio Features.csv':
0                                                   []
1                                          ['novelty']
2    ['adult standards', 'brill building pop', 'eas...
3     ['rock-and-roll', 'space age pop', 'surf music']
4                ['dance pop', 'pop', 'post-teen pop']
Name: spotify_genre, dtype: object


In [4]:
import ast

# Drop NaN values to avoid errors
genre_series = audio_features_df['spotify_genre'].dropna()

# Parse genre strings to lists or empty list if NaN
audio_features_df['parsed_genres'] = audio_features_df['spotify_genre'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else []
)

# Drop rows where the parsed list is empty
audio_features_df = audio_features_df[audio_features_df['parsed_genres'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

# (Optional) Reset index
audio_features_df.reset_index(drop=True, inplace=True)

# Convert stringified lists to real Python lists using ast.literal_eval
parsed_genres = genre_series.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Flatten all lists into one
all_genres = [genre for sublist in parsed_genres for genre in sublist]

# Get unique genres
unique_genres = sorted(set(all_genres))

# Display results
print(f"🎵 Total unique genres: {len(unique_genres)}")
print(unique_genres)


🎵 Total unique genres: 1145
['21st century classical', 'a cappella', 'acid house', 'acid jazz', 'acoustic blues', 'acoustic pop', 'acoustic punk', 'adult standards', 'afro house', 'afrobeat', 'afropop', 'alabama indie', 'alabama metal', 'alabama rap', 'alaska indie', 'alberta country', 'album rock', 'albuquerque indie', 'alt z', 'alt-idol', 'alternative americana', 'alternative country', 'alternative dance', 'alternative emo', 'alternative hip hop', 'alternative metal', 'alternative pop', 'alternative pop rock', 'alternative r&b', 'alternative rock', 'alternative roots rock', 'ambient folk', 'ambient psychill', 'american folk revival', 'american post-rock', 'anarcho-punk', 'anime', 'ann arbor indie', 'anthem emo', 'anthem worship', 'anti-folk', 'antiviral pop', 'appalachian folk', 'argentine rock', 'arkansas country', 'arkansas hip hop', 'art pop', 'art rock', 'asmr', 'athens indie', 'atl hip hop', 'atl trap', 'audiophile vocal', 'aussietronica', 'austindie', 'australian alternative ro

In [6]:
from collections import Counter

genre_map = {
    'novelty': 'Pop',
    'idol': 'Pop',
    'boy band': 'Pop',
    'broadway': 'Pop',
    'classic girl group': 'Pop',
    'calming instrumental': 'Easy Listening',
    'neo mellow': 'Pop',
    'urban contemporary': 'R&B',

    'motown': 'R&B',
    'deep motown': 'R&B',
    'rare groove': 'R&B',
    'new jack swing': 'R&B',

    'comic': 'Comedy',
    'comedy': 'Comedy',
    'parody': 'Comedy',

    'hollywood': 'Soundtrack',
    'movie tunes': 'Soundtrack',
    'show tunes': 'Soundtrack',
    'christmas instrumental': 'Holiday',

    'nashville sound': 'Country',
    'norteno': 'Regional Mexican',
    'beach music': 'Soul',

    'latin': 'Latin',
    'tropical': 'Latin',
    'brooklyn drill': 'Hip Hop',

    'merseybeat': 'Rock',
    'british invasion': 'Rock',
    'adult standards': 'Easy Listening',
    'easy listening': 'Easy Listening',
    'deep adult standards': 'Easy Listening',
    'mellow gold': 'Easy Listening',
    'lounge': 'Easy Listening',
    
    'freestyle': 'Dance',
    'disco': 'Dance',
    'hi-nrg': 'Dance',
    'miami bass': 'Dance',

    'quiet storm': 'R&B',
    'doo-wop': 'R&B',
    'vocal harmony group': 'R&B',

    'surf music': 'Rock',
    'cowboy western': 'Country',
    'bakersfield sound': 'Country',

    'ska': 'Ska',
    'ska punk': 'Ska',
    'ska revival': 'Ska',
    'traditional ska': 'Ska',
    'german ska': 'Ska',
    'pop': 'Pop',
    'rock': 'Rock',
    'hip hop': 'Hip Hop',
    'rap': 'Hip Hop',
    'r&b': 'R&B',
    'edm': 'Electronic',
    'electro': 'Electronic',
    'house': 'Electronic',
    'dance': 'Dance',
    'metal': 'Metal',
    'jazz': 'Jazz',
    'classical': 'Classical',
    'country': 'Country',
    'folk': 'Folk',
    'reggae': 'Reggae',
    'trap': 'Trap',
    'blues': 'Blues',
    'indie': 'Indie',
    'soul': 'Soul',
    'funk': 'Funk'
}
# Function to assign a broad genre
def map_to_broad_genre(genre_list, genre_map):
    if not isinstance(genre_list, list):
        return ['Other']
    
    matched = []
    for genre in genre_list:
        for key in genre_map:
            if key in genre.lower():
                matched.append(genre_map[key])
                break  # Avoid multiple matches per input genre

    return matched if matched else ['Other']


# Parse the spotify_genre column into actual lists
audio_features_df['parsed_genres'] = audio_features_df['spotify_genre'].dropna().apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Assign each row a broad genre
audio_features_df['broad_genre'] = audio_features_df['parsed_genres'].apply(
    lambda genres: map_to_broad_genre(genres, genre_map))

# Show genre distribution
genre_counts = audio_features_df['broad_genre'].value_counts()
print("🎼 Broad genre distribution:")
print(genre_counts)

#  Identify and analyze the "Other" genres
other_rows = audio_features_df[audio_features_df['broad_genre'] == 'Other']
other_genres = [
    genre for sublist in other_rows['parsed_genres'] if isinstance(sublist, list) for genre in sublist
]
other_genre_counts = Counter(other_genres)

# Save unmatched genres to CSV
other_genre_df = pd.DataFrame(
    other_genre_counts.items(), columns=['genre', 'count']
).sort_values(by='count', ascending=False)

other_genre_df.to_csv("unmatched_other_genres.csv", index=False)
audio_features_df.to_csv("broad_genres.csv", index=False)



🎼 Broad genre distribution:
broad_genre
[Pop]                                                           807
[Pop, Pop]                                                      737
[Pop, Pop, Pop]                                                 557
[Rock]                                                          404
[Country, Country, Country, Rock]                               399
                                                               ... 
[Rock, Pop, Rock, Pop, Pop, Rock]                                 1
[Pop, Electronic, Electronic, Pop, Electronic, Latin]             1
[Jazz, Jazz, Funk, Jazz, Jazz, Jazz, Jazz, Jazz, Soul, Jazz]      1
[Folk, Rock, Rock, Folk]                                          1
[Rock, Pop, Pop, Rock, Pop, Rock]                                 1
Name: count, Length: 1541, dtype: int64


In [8]:
# Flatten all lists in 'broad_genre' into one list
all_broad_genres = [
    genre
    for genre_list in audio_features_df['broad_genre']
    if isinstance(genre_list, list)
    for genre in genre_list
]

# Get unique genres
unique_broad_genres = sorted(set(all_broad_genres))

# Print results
print(f"🎵 Found {len(unique_broad_genres)} unique broad genres:")
print(unique_broad_genres)

🎵 Found 24 unique broad genres:
['Blues', 'Classical', 'Comedy', 'Country', 'Dance', 'Easy Listening', 'Electronic', 'Folk', 'Funk', 'Hip Hop', 'Holiday', 'Indie', 'Jazz', 'Latin', 'Metal', 'Other', 'Pop', 'R&B', 'Reggae', 'Regional Mexican', 'Rock', 'Ska', 'Soul', 'Soundtrack']


In [11]:
# Normalize all genres in the 'broad_genre' list to lowercase
audio_features_df['broad_genre'] = audio_features_df['broad_genre'].apply(
    lambda genres: [g.lower() for g in genres] if isinstance(genres, list) else genres
)

In [12]:
# Define genres (in lowercase)
target_genres = [
    'blues', 'classical', 'comedy', 'country', 'dance', 'easy listening',
    'electronic', 'folk', 'funk', 'hip hop', 'holiday', 'indie', 'jazz',
    'latin', 'metal', 'other', 'pop', 'r&b', 'reggae', 'regional mexican',
    'rock', 'ska', 'soul', 'soundtrack'
]

# Create a dict to store each genre-specific DataFrame
genre_dfs = {}

for genre in target_genres:
    genre_dfs[genre] = audio_features_df[
        audio_features_df['broad_genre'].apply(lambda g: genre in g if isinstance(g, list) else False)
    ]

# ✅ Example: show a few rows from the 'pop' genre
print("🎧 Example rows for 'pop' genre:")
print(genre_dfs['pop'].head())



🎧 Example rows for 'pop' genre:
   index                                             SongID       Performer  \
0      1  ¿Dònde Està Santa Claus? (Where Is Santa Claus...      Augie Rios   
1      2             ......And Roses And RosesAndy Williams   Andy Williams   
2      3           ...And Then There Were DrumsSandy Nelson    Sandy Nelson   
3      4                ...Baby One More TimeBritney Spears  Britney Spears   
4      5                       ...Ready For It?Taylor Swift    Taylor Swift   

                                               Song  \
0  ¿Dònde Està Santa Claus? (Where Is Santa Claus?)   
1                         ......And Roses And Roses   
2                      ...And Then There Were Drums   
3                             ...Baby One More Time   
4                                  ...Ready For It?   

                                       spotify_genre        spotify_track_id  \
0                                        ['novelty']                     NaN   
1 

In [13]:
import os

# Step 1: Define output folder
output_folder = "broad_genre_csvs"
os.makedirs(output_folder, exist_ok=True)  # create folder if it doesn't exist

# Step 2: Save each genre's DataFrame to a CSV
for genre, df in genre_dfs.items():
    filename = f"{genre.replace(' ', '_')}_songs.csv"  # replace spaces for clean filenames
    filepath = os.path.join(output_folder, filename)
    df.to_csv(filepath, index=False)

print(f"✅ All genre DataFrames saved in '{output_folder}' folder.")


✅ All genre DataFrames saved in 'broad_genre_csvs' folder.


In [16]:
# Updated list of features
features_to_correlate = ['danceability', 'energy', 'valence', 'tempo', 'spotify_track_popularity']

# Dictionary to store correlation matrices
genre_correlations = {}

# Compute correlation matrix for each genre
for genre, df in genre_dfs.items():
    if all(feature in df.columns for feature in features_to_correlate):
        corr_matrix = df[features_to_correlate].corr()
        genre_correlations[genre] = corr_matrix

# ✅ Example: show the correlation matrix for 'pop'
print("🎯 Correlation matrix for 'pop':")
print(genre_correlations['pop'])


🎯 Correlation matrix for 'pop':
                          danceability    energy   valence     tempo  \
danceability                  1.000000  0.222903  0.337454 -0.134474   
energy                        0.222903  1.000000  0.357882  0.152557   
valence                       0.337454  0.357882  1.000000  0.067511   
tempo                        -0.134474  0.152557  0.067511  1.000000   
spotify_track_popularity      0.241946  0.168633 -0.235468  0.012445   

                          spotify_track_popularity  
danceability                              0.241946  
energy                                    0.168633  
valence                                  -0.235468  
tempo                                     0.012445  
spotify_track_popularity                  1.000000  


In [17]:
import os

# Step 1: Create output folder
output_folder = "correlation_matrix"
os.makedirs(output_folder, exist_ok=True)

# Step 2: Save each correlation matrix
for genre, corr_matrix in genre_correlations.items():
    filename = f"{genre.replace(' ', '_')}_correlation.csv"
    filepath = os.path.join(output_folder, filename)
    corr_matrix.to_csv(filepath)

print(f"✅ All correlation matrices saved in '{output_folder}' folder.")


✅ All correlation matrices saved in 'correlation_matrix' folder.


In [18]:
# List and print all files in the correlation_matrix folder
saved_files = sorted(os.listdir("correlation_matrix"))

print("📂 Correlation matrix files saved:")
for filename in saved_files:
    print(f"- {filename}")

📂 Correlation matrix files saved:
- blues_correlation.csv
- classical_correlation.csv
- comedy_correlation.csv
- country_correlation.csv
- dance_correlation.csv
- easy_listening_correlation.csv
- electronic_correlation.csv
- folk_correlation.csv
- funk_correlation.csv
- hip_hop_correlation.csv
- holiday_correlation.csv
- indie_correlation.csv
- jazz_correlation.csv
- latin_correlation.csv
- metal_correlation.csv
- other_correlation.csv
- pop_correlation.csv
- r&b_correlation.csv
- reggae_correlation.csv
- regional_mexican_correlation.csv
- rock_correlation.csv
- ska_correlation.csv
- soul_correlation.csv
- soundtrack_correlation.csv


In [22]:
import pandas as pd

# Initialize list to store rows
correlation_summary = []

# For each genre, extract specific correlation values
for genre, corr in genre_correlations.items():
    row = {'broad_genre': genre}
    # Loop through all pairs (upper triangle only)
    for i, col1 in enumerate(corr.columns):
        for j, col2 in enumerate(corr.columns):
            if i < j:  # to avoid duplicates and self-correlation
                col_name = f"{col1}_{col2}_corr"
                row[col_name] = corr.loc[col1, col2]
    correlation_summary.append(row)

# Convert to DataFrame
correlation_df = pd.DataFrame(correlation_summary)

# Optional: Sort by genre name
correlation_df.sort_values(by='broad_genre', inplace=True)

# Show preview
print("📊 Summary of pairwise correlations by genre:")
print(correlation_df.head())


📊 Summary of pairwise correlations by genre:
  broad_genre  danceability_energy_corr  danceability_valence_corr  \
0       blues                  0.080681                   0.508223   
1   classical                  0.686095                   0.686683   
2      comedy                 -0.007757                   0.530567   
3     country                  0.191059                   0.477868   
4       dance                  0.352470                   0.572553   

   danceability_tempo_corr  danceability_spotify_track_popularity_corr  \
0                -0.183001                                    0.086516   
1                -0.091949                                   -0.083237   
2                -0.195761                                    0.113142   
3                -0.185238                                    0.100877   
4                -0.199731                                    0.097867   

   energy_valence_corr  energy_tempo_corr  \
0             0.447260           0.174577   

In [24]:
# Rename all column names in correlation_df that contain 'spotify_track_popularity'
correlation_df.columns = [
    col.replace('spotify_track_popularity', 'popularity') for col in correlation_df.columns
]

# Preview to confirm
print("✅ Renamed columns in correlation_df:")
print(correlation_df.columns.tolist())


✅ Renamed columns in correlation_df:
['broad_genre', 'danceability_energy_corr', 'danceability_valence_corr', 'danceability_tempo_corr', 'danceability_popularity_corr', 'energy_valence_corr', 'energy_tempo_corr', 'energy_popularity_corr', 'valence_tempo_corr', 'valence_popularity_corr', 'tempo_popularity_corr', 'avg_danceability', 'avg_energy', 'avg_valence', 'avg_tempo', 'avg_popularity']


In [25]:
# List of features to compute the mean for
features_to_average = ['danceability', 'energy', 'valence', 'tempo', 'spotify_track_popularity']

# Create a DataFrame to hold mean values per genre
mean_values = []

for genre, df in genre_dfs.items():
    means = {'broad_genre': genre}
    for feature in features_to_average:
        if feature in df.columns:
            means[f'avg_{feature}'] = df[feature].mean()
    mean_values.append(means)

# Convert to DataFrame
mean_df = pd.DataFrame(mean_values)

# ✅ Merge the means into the correlation summary
correlation_df = pd.merge(correlation_df, mean_df, on='broad_genre', how='left')

correlation_df.sort_values(by='broad_genre', inplace=True)
correlation_df.to_csv("broad_genre_correlation_summary.csv", index=False)

